In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from obspy import read
from obspy.signal.trigger import classic_sta_lta, trigger_onset
from scipy import signal
import os
from datetime import datetime, timedelta

# Définir les répertoires
data_directory = 'data/mars/test/data/'  # Chemin vers le dossier contenant les fichiers CSV et MiniSEED
output_directory = 'output/'  # Répertoire pour sauvegarder les résultats

# Créer le répertoire de sortie s'il n'existe pas
os.makedirs(output_directory, exist_ok=True)

# Obtenir la liste des fichiers CSV
csv_files = [f for f in os.listdir(data_directory) if f.endswith('.csv')]
print(csv_files)

def butter_highpass_filter(data, cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    y = signal.filtfilt(b, a, data)
    return y

for csv_file in csv_files:
    # ... (Votre code existant jusqu'à la lecture du fichier mseed) ...
    cat_directory = './data/mars/test/data'
    cat_file = cat_directory + '/' + csv_file
    cat = pd.read_csv(cat_file)
    
    # Conversion de la colonne 'time_abs' en datetime
    cat['arrival_time'] = pd.to_datetime(cat['time_abs(%Y-%m-%dT%H:%M:%S.%f)'], format='%Y-%m-%dT%H:%M:%S.%f')
    
    csv_file = f'{cat_file}'
    mseed_file = csv_file.replace('csv', 'mseed')
    st = read(mseed_file)
    tr = st.traces[0].copy()
    tr_times = tr.times()
    tr_data = tr.data
    starttime = tr.stats.starttime.datetime
    
    # Utiliser le premier temps d'arrivée (si disponible)
    if not cat['arrival_time'].empty:
        arrival_time = cat['arrival_time'].iloc[0]
        arrival = (arrival_time - starttime).total_seconds()
    else:
        arrival = 0  # Fallback si aucun temps d'arrivée n'est disponible

    starttime = tr.stats.starttime.datetime
    arrival = (arrival_time - starttime).total_seconds()

    # Calcul du rapport signal sur bruit (SNR)
    signal_power = np.mean(tr_data ** 2)
    noise_power = np.mean((tr_data[triggers[1]:] - np.mean(tr_data[triggers[1]:])) ** 2)  # SNR après le premier événement
    snr = signal_power / noise_power

    # Appliquer le filtre passe-haut
    fs = tr.stats.sampling_rate  # Fréquence d'échantillonnage
    cutoff = 1  # Fréquence de coupure en Hz (ajustez selon vos besoins)
    filtered_data = butter_highpass_filter(tr_data, cutoff, fs)

    # Utiliser les données filtrées pour le reste du traitement
    tr_data = filtered_data

    # Initialize figure pour les données filtrées
    fig, ax = plt.subplots(1, 1, figsize=(10, 3))
    ax.plot(tr_times, tr_data)
    ax.axvline(x=arrival, color='red', label='Rel. Arrival')
    ax.legend(loc='upper left')
    ax.set_xlim([min(tr_times), max(tr_times)])
    ax.set_ylabel('Filtered Velocity (c/s)')
    ax.set_xlabel('Time (s)')
    ax.set_title(f'Filtered {mseed_file}', fontweight='bold')
    plt.show()

    # Initialize figure
    fig, ax = plt.subplots(1, 1, figsize=(10, 3))
    ax.plot(tr_times, tr_data)
    ax.axvline(x=arrival, color='red', label='Rel. Arrival')
    ax.legend(loc='upper left')
    ax.set_xlim([min(tr_times), max(tr_times)])
    ax.set_ylabel('Velocity (c/s)')
    ax.set_xlabel('Time (s)')
    ax.set_title(f'{mseed_file}', fontweight='bold')

    # Sampling frequency of our trace
    df = tr.stats.sampling_rate

    # How long should the short-term and long-term window be, in seconds?
    sta_len = 120
    lta_len = 600

    cft = classic_sta_lta(tr_data, int(sta_len * df), int(lta_len * df))

    # Plot characteristic function
    fig, ax = plt.subplots(1, 1, figsize=(12, 3))
    ax.plot(tr_times, cft)
    ax.set_xlim([min(tr_times), max(tr_times)])
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Characteristic function')
    ax.set_title(f'{mseed_file}', fontweight='bold')

    # Définition des seuils de déclenchement
    thr_on = 4
    thr_off = 1.5
    on_off = np.array(trigger_onset(cft, thr_on, thr_off))

    # Création du graphique
    fig, ax = plt.subplots(1, 1, figsize=(12, 3))
    ax.plot(tr_times, tr_data, label='Sismogramme')

    # Tracé des déclencheurs
    event_times = []
    amplitudes = []
    on_label_used = False
    off_label_used = False
    for i in range(len(on_off)):
        triggers = on_off[i]
        
        # Déclencheur "on" (début de l'événement)
        event_start = tr_times[triggers[0]]
        event_end = tr_times[triggers[1]]
        
        ax.axvline(x=event_start, color='red', label='Début événement' if not on_label_used else '')
        ax.axvline(x=event_end, color='purple', label='Fin événement' if not off_label_used else '')
        
        event_times.append((event_start, event_end))
        amplitudes.append(np.max(tr_data[triggers[0]:triggers[1]]))  # Amplitude max de l'événement

        on_label_used = True
        off_label_used = True

    ax.set_xlim([min(tr_times), max(tr_times)])
    ax.set_xlabel('Temps (s)')
    ax.set_ylabel('Amplitude')
    ax.legend()
    plt.title('Sismogramme avec détections STA/LTA')
    plt.show()

    # Calcul du SNR après filtrage
    signal_power = np.mean(tr_data[triggers[0]:triggers[1]] ** 2)
    noise_power = np.mean((tr_data[:triggers[0]] - np.mean(tr_data[:triggers[0]])) ** 2)
    snr = 10 * np.log10(signal_power / noise_power)

    # Résumé des résultats
    summary_df = pd.DataFrame({
        'Event Start (s)': [event[0] for event in event_times],
        'Event End (s)': [event[1] for event in event_times],
        'Amplitude': amplitudes,
        'SNR': [snr] * len(event_times)
    })

    # Résumé des résultats
    summary_df = pd.DataFrame({
        'Event Start (s)': [event[0] for event in event_times],
        'Event End (s)': [event[1] for event in event_times],
        'Amplitude': amplitudes,
        'SNR': [snr] * len(event_times)
    })

    # Sauvegarder le résumé dans un fichier CSV
    summary_file = f'{mseed_file}_summary.csv'
    summary_df.to_csv(summary_file, index=False)

    print(f"Résumé sauvegardé dans : {summary_file}")

['XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv', 'XB.ELYSE.02.BHV.2021-05-02HR01_evid0017.csv', 'XB.ELYSE.02.BHV.2022-04-09HR22_evid0002.csv', 'XB.ELYSE.02.BHV.2019-09-21HR03_evid0032.csv', 'XB.ELYSE.02.BHV.2021-10-11HR23_evid0011.csv', 'XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv', 'XB.ELYSE.02.BHV.2022-05-04HR23_evid0001.csv', 'XB.ELYSE.02.BHV.2021-12-24HR22_evid0007.csv', 'XB.ELYSE.02.BHV.2019-07-26HR12_evid0034.csv']


KeyError: 'time_abs(%Y-%m-%dT%H:%M:%S.%f)'